In [5]:
import math
import numpy as np
import scipy as sp
from scipy import linalg
import Euler_Angles


L1 = 0.14974
L2 = 0.015
L3 = 0.1812
L4 = 0
L5 = 0.150
L6 = 0.0695
L7 = 0.0303

p = np.array([0,0,0,1])
v0 = np.array([[1],[0],[0],[0]])
v1 = np.array([[0],[1],[0],[0]])
v2 = np.array([[0],[0],[1],[0]])


def transX(th, x, y, z):
    s = math.sin(th)
    c = math.cos(th)
    return np.array([[1, 0, 0, x], [0, c, -s, y], [0, s, c, z], [0, 0, 0, 1]])

def transY(th, x, y, z):
    s = math.sin(th)
    c = math.cos(th)
    return np.array([[c, 0, -s, x], [0, 1, 0, y], [s, 0, c, z], [0, 0, 0, 1]])

def transZ(th, x, y, z):
    s = math.sin(th)
    c = math.cos(th)
    return np.array([[c, -s, 0, x], [s, c, 0, y], [0, 0, 1, z], [0, 0, 0, 1]])

def calc_trans_matrix(angles):
    
    T1 = transY(-angles[0], 0, L1, 0)
    T2 = transZ(angles[1], 0, 0, 0)
    Td = transY(9.0/180.0*math.pi, L3, L2, 0)
    T3 = transX(angles[2], 0, 0, 0)
    T4 = transZ(angles[3], 0, 0, 0)
    T5 = transX(angles[4], L5, 0, 0)
    T6 = transZ(0, L6, 0, -L7)
    
    T1Abs = T1
    T2Abs = T1Abs.dot(T2)
    TdAbs = T2Abs.dot(Td)
    T3Abs = TdAbs.dot(T3)
    T4Abs = T3Abs.dot(T4)
    T5Abs = T4Abs.dot(T5)
    T6Abs = T5Abs.dot(T6)
    
    return T1Abs, T2Abs, T3Abs, T4Abs, T5Abs, T6Abs

def calc_fk_and_jacob(angles, jacob=True, right=True):
    _L1_ = -L1 if right else L1
    _L2_ = -L2 if right else L2

    T1 = transY(-angles[0], 0, _L1_, 0)
    T2 = transZ(angles[1], 0, 0, 0)
    Td = transY(9.0/180.0*math.pi, L3, _L2_, 0)
    T3 = transX(angles[2], 0, 0, 0)
    T4 = transZ(angles[3], 0, 0, 0)
    T5 = transX(angles[4], L5, 0, 0)
    T6 = transZ(0, L6, 0, -L7)
    
    T1Abs = T1
    T2Abs = T1Abs.dot(T2)
    TdAbs = T2Abs.dot(Td)
    T3Abs = TdAbs.dot(T3)
    T4Abs = T3Abs.dot(T4)
    T5Abs = T4Abs.dot(T5)
    T6Abs = T5Abs.dot(T6)

    pos = T6Abs.dot(p)
    ori = T6Abs[0:3,0:3]

    if not jacob:
        return pos, ori

    OfstT1 = L1 * T1Abs.dot(v1)
    OfstTd = TdAbs.dot(np.array([[L3], [L2], [0], [0]]))
    OfstT5 = L5 * T5Abs.dot(v0)
    OfstT6 = T5Abs.dot(np.array([[L6], [0], [-L7], [0]]))

    vec6 = OfstT6
    vec5 = vec6 + OfstT5
    vec4 = vec5
    vec3 = vec4
    vecd = vec3 + OfstTd
    vec2 = vecd
    vec1 = vec2 + OfstT1
    
    j1 = T1Abs.dot(v1)
    j2 = T2Abs.dot(v2)
    jd = TdAbs.dot(v1)
    j3 = T3Abs.dot(v0)
    j4 = T4Abs.dot(v2)
    j5 = T5Abs.dot(v0)
    
    J1 = cross(j1, vec1)
    J2 = cross(j2, vec2)
    J3 = cross(j3, vec3)
    J4 = cross(j4, vec4)
    J5 = cross(j5, vec5)
    
    Jv = np.c_[J1, J2, J3, J4, J5]
    
    jw = np.c_[j1, j2, j3, j4, j5][0:3,:]

    J = np.vstack(Jv, Jw)
    
    euler_ori = np.array(Euler_Angles.rot2euler(ori))
    
    return pos, euler_ori, J


def cross(j, v):
    t0 = j[1][0] * v[2][0] - j[2][0] * v[1][0]
    t1 = j[2][0] * v[0][0] - j[0][0] * v[2][0]
    t2 = j[0][0] * v[1][0] - j[1][0] * v[0][0]
    return np.array([[t0], [t1], [t2]])
        


In [2]:
import numpy as np
np.vstack

In [4]:
import math
import numpy as np
import scipy as sp
from scipy import linalg

import forward_kinematics as fk

def calc_inv_pos(angles, target_pos, target_ori, epsilon, right=True):
    p  = np.array([0,0,0,1])
    angs = np.array([a for a in angles])
    sum_old = 100000
    while True:
        pos, ori, J = fk.calc_fk_and_jacob(angs, jacob=True, right=right)
        J = _calc_invJ(j)
        delta_pos = np.matrix((target_pos-pos)[0:3]).transpose()
        delta_ori = np.matrix((target_ori-ori)[0:3]).transpose()
        delta = np.vstack((delta_pos, delta_ori))
        
        v_w = (J * delta).transpose()
        angs = np.squeeze(np.asarray(v_w)) + angs
        
        sum = 0
        for d in delta:
            sum = sum + math.fabs(d)
        #sum = np.sum(delta_pos)
        if sum < epsilon:
            break
        if sum > sum_old:
            print '# set_position error : Distance can not converged.'
            return None
        sum_old = sum
    return angs

def _calc_invJ(J, epsilon = 0.01):
    u, sigma, v = np.linalg.svd(J, full_matrices=True)
    sigma_ = [1/s if s > epsilon else 0 for s in sigma]
    rank_v = np.shape(J)[0]
    rank_h = np.shape(J)[1]

    return np.matrix(v.transpose()) * np.matrix(linalg.diagsvd(sigma_, rank_h, rank_v)) * np.matrix(u.transpose())